# Adiabatic SSM model

## Finite-horizon prediction accuracy

This notebook quantifies the predictive power of adiabatic SSM models by doing finite-horizon predictions across the robot's workspace and evaluating the corresponding prediction errors.

In [1]:
from os import listdir
from os.path import join, split, isdir
import pickle
import yaml
import numpy as np
from numpy.random import randint
from tqdm.auto import tqdm
import time
import yaml
np.set_printoptions(linewidth=100)

In [2]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot
from interpolators import InterpolatorFactory

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt

In [4]:
# Adiabatic SSM settings
ROMOrder = 3
# N_samples = 100
DT = 0.01
INTERPOLATE = "reduced_coords" # "xyz" # "xy" # "reduced_coords"

Trunk settings

In [5]:
observables = "delay-embedding" # "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = join(robot_dir, 'rest_qv.pkl')

# load rest position
with open(rest_file, 'rb') as f:
    rest_data = pickle.load(f)
    rest_q = rest_data['q'][0]

In [6]:
model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms_N=100_sparsity=0.95" # 9" # 147" # 33_handcrafted" #  # 
model_names = [name for name in sorted(listdir(model_dir)) if isdir(join(model_dir, name))]
print(model_names)

USE_DEFAULT_MODELS = True

default_model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms_N=9"
default_model_names = [name for name in sorted(listdir(default_model_dir)) if isdir(join(default_model_dir, name))]
print(default_model_names)

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099']
['000', '001', '002', '003', '004', '005', '006', '007', '008']
N_models: 109


In [7]:
filter_models = "only_default" # "sparsify" # "outlier_removal" # "all" # "test_results" # "fixed_grid"

Compute observables (delay embedding)

In [8]:
if observables == "delay-embedding":
    N_DELAY = 4
    # observables are position of tip + n_delay delay embeddings of the tip position
    assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)
elif observables == "pos-vel":
    # observables is position and velocity of tip node
    def assemble_observables(oData):
        if oData.shape[0] > 6:
            tip_node_slice = np.s_[3*TIP_NODE:3*TIP_NODE+3]
        else:
            tip_node_slice = np.s_[:3]
        return np.vstack((oData[tip_node_slice, :], np.gradient(oData[tip_node_slice, :], DT, axis=1)))

Load local SSM models

In [12]:
models = []
test_results = []

if USE_DEFAULT_MODELS:
    for model_name in default_model_names:
        dir = join(default_model_dir, model_name, f"SSMmodel_{observables}_ROMOrder=3_globalV_fixed-delay")
        with open(join(dir, "SSM_model.pkl"), "rb") as f:
            model = pickle.load(f)
            models.append(model)
        with open(join(dir, "test_results.yaml"), "rb") as f:
            test_results_dict = yaml.safe_load(f)
            test_results.append([test_results_dict['like training data']['RMSE'], test_results_dict['open-loop_circle']['RMSE']])

if filter_models != "only_default":
    for model_name in model_names:
        dir = join(model_dir, model_name, f"SSMmodel_{observables}_globalV")
        with open(join(dir, "SSM_model.pkl"), "rb") as f:
            model = pickle.load(f)
            models.append(model)
        with open(join(dir, "test_results.yaml"), "rb") as f:
            test_results_dict = yaml.safe_load(f)
            test_results.append([test_results_dict['like training data']['RMSE'], test_results_dict['open-loop_circle']['RMSE']])

V = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
v_coeff = [model['model']['v_coeff'] for model in models]
B_r = [model['model']['B'] for model in models]
q_bar = [(model['model']['q_eq'] - rest_q)[TIP_NODE*3:TIP_NODE*3+3] for model in models]
u_bar = [model['model']['u_eq'] for model in models]
ROMOrder = models[0]['params']['ROM_order']
SSMOrder = models[0]['params']['SSM_order']
for model in models:
    assert model['params']['ROM_order'] == ROMOrder
    assert model['params']['SSM_order'] == SSMOrder
test_results = np.array(test_results)

N_models = len(V)
print("N_models:", N_models)

N_models: 9


In [13]:
if INTERPOLATE == "xyz":
    psi_eq = [q_bar[i][:3] for i in range(N_models)]
elif INTERPOLATE == "xy":
    psi_eq = [q_bar[i][:2] for i in range(N_models)]
elif INTERPOLATE == "reduced_coords":
    psi_eq = [V[i].T @ np.tile(q_bar[i], 5) for i in range(N_models)]
# psi_eq
# print(u_bar)
# print(np.sum([u == 0 for u in u_bar], axis=1))
# print(np.mean(np.sum([u == 0 for u in u_bar], axis=1)))
print(psi_eq)

[array([ 0.0116741 ,  0.09136295,  0.0383846 ,  0.00076458, -0.00433124,  0.00074858]), array([ 72.9267292 , -20.04814538, -28.61369534,   3.39920343,   1.74130791,  -1.4508973 ]), array([-32.36741758, -62.47260605, -37.32622321,  -1.50355753,   2.99363918,  -1.87813233]), array([-73.89207721,  33.43785334,   3.89146107,  -3.11097681,  -1.89676717,  -1.53847375]), array([29.37044068, 73.49785929, 12.97434427,  1.68149513, -3.06984472, -1.00990627]), array([ 41.81167278, -82.87095552, -66.49121622,   1.95456372,   4.76607361,  -3.35980064]), array([-108.11923851,  -27.69264586,  -34.11427275,   -4.68187634,    1.04114646,   -3.55186094]), array([-44.18260968, 106.98101026,  15.62895123,  -1.40198947,  -4.94621196,  -2.6718042 ]), array([101.23388332,  54.22021898, -16.27941313,   5.04466878,  -1.35641969,  -2.5637134 ])]


In [14]:
plt.close("all")
fig = plt.figure()

R_linear_part = [r[:, :6] for r in r_coeff]
eigenvalues_real_part = [np.real(np.linalg.eigvals(r)) for r in R_linear_part]
metric = [np.linalg.norm(eigenvalues, ord=-np.inf) for eigenvalues in eigenvalues_real_part]

psi_eq_array = np.array(psi_eq)
color_by_metric = False

if INTERPOLATE == "xyz":
    ax = plt.axes(projection="3d")
    if color_by_metric:
        sc = ax.scatter(psi_eq_array[:, 0], psi_eq_array[:, 1], psi_eq_array[:, 2], marker="o", c=metric, vmax=None, cmap='viridis')
    else:
        sc = ax.scatter(psi_eq_array[:, 0], psi_eq_array[:, 1], psi_eq_array[:, 2], marker="o", color="darkblue")
    for i in range(N_models):
        ax.text(psi_eq_array[i, 0], psi_eq_array[i, 1], psi_eq_array[i, 2], str(i), size=10, zorder=1, color='k')
    ax.set_zlabel(r"$z$ [mm]")
else:
    ax = plt.axes()
    if INTERPOLATE == "xy":
        color = "tab:blue"
    elif INTERPOLATE == "reduced_coords":
        color = "tab:orange"
    if color_by_metric:
        sc = ax.scatter(psi_eq_array[:, 0], psi_eq_array[:, 1], marker="o", c=metric, vmax=None, cmap="viridis")
    else:
        sc = ax.scatter(psi_eq_array[:, 0], psi_eq_array[:, 1], marker="o", color=color)
if INTERPOLATE in  ["xy", "xyz"]:
    ax.set_xlabel(r"$x$ [mm]")
    ax.set_ylabel(r"$y$ [mm]")
elif INTERPOLATE == "reduced_coords":
    ax.set_xlabel(r"$x_1$")
    ax.set_ylabel(r"$x_2$")
ax.set_aspect("equal")
if color_by_metric:
    plt.colorbar(sc, ax=ax, label=r"$||Re(\lambda)||_{\infty}$")
    fig.suptitle("Slowest decay mode of the ROM's linear part")
fig.savefig(join(model_dir, f"models_in_{INTERPOLATE}.svg"), dpi=300)
fig.show()

Sparsify grid: Start with origin model and add more models while ensuring that each model is the only one inside a ball with size $s$

In [15]:
if filter_models == "sparsify":
    radius = 10
    use_models = [0]
    for model_name_i in model_names:
        model_i = int(model_name_i)
        add_model = True
        for model_j in use_models:
            distance = np.linalg.norm(psi_eq[model_i] - psi_eq[model_j])
            if distance < radius:
                add_model = False
                break
        if add_model:
            use_models.append(model_i)

Fix regular grid. Around each grid point, pick the available model with minimum $|u|$

In [16]:
if filter_models == "fixed_grid":
    ranges = {
        'x': [-45, 45],
        'y': [-45, 45],
        'z': [0, -20]
    }
    n_grid = {
        'x': 7,
        'y': 7,
        'z': 3,
    }
    x = np.linspace(ranges['x'][0], ranges['x'][1], n_grid['x'])
    y = np.linspace(ranges['y'][0], ranges['y'][1], n_grid['y'])
    z = np.linspace(ranges['z'][0], ranges['z'][1], n_grid['z'])
    use_models = [0]
    radius = 10
    for zi in z:
        for yi in y:
            for xi in x:
                if xi == 0 and yi == 0 and zi == 0:
                    continue
                # potential_models = []
                # # find the models within a ball with radius 10mm around the point
                # # choose model with smallest |u|
                # for model_name_i in model_names:
                #     model_i = int(model_name_i)
                #     distance = np.linalg.norm([xi, yi, zi] - eq_q[model_i])
                #     if distance < radius and model_i not in use_models:
                #         potential_models.append(model_i)
                # if potential_models:
                #     use_models.append(min(potential_models, key=(lambda id: np.linalg.norm(u_bar[id]))))
                min_dist_model = min([int(model_name) for model_name in model_names], key=(lambda id: np.linalg.norm(psi_eq[id] - [xi, yi, zi])))
                if min_dist_model not in [int(model_name) for model_name in use_models]:
                    use_models.append(min_dist_model)

Filter out models based on test results

In [17]:
if filter_models == "test_results":
    print(test_results[:, 0])
    good_models = ~np.isnan(test_results[:, 0]) & (test_results[:, 0] < 0.3)
    use_models = [int(model_name) for model_name in model_names if good_models[int(model_name)]]

Filter out models with large pre-tensionings

In [18]:
if filter_models == "small_pretensionings":
    bad_models = (np.linalg.norm(np.array(u_bar), axis=1) > 1000)
    use_models = [int(model_name) for model_name in model_names if not bad_models[int(model_name)]]

In [19]:
if filter_models == "outlier_removal":

    ref_model = 0 # origin model
    compare_coeffs = [r_coeff]

    ref_coeff_array = np.concatenate([coeff[ref_model].flatten() for coeff in compare_coeffs])

    use_models = [ref_model]

    for i in range(N_models):
        if i == ref_model:
            continue
        coeff_array = np.concatenate([coeff[i].flatten() for coeff in compare_coeffs])
        rel_coeff_distance = np.abs(1 - (coeff_array / ref_coeff_array))
        if np.max(rel_coeff_distance) < 10. * np.linalg.norm(q_bar[i] - q_bar[ref_model]):
            use_models.append(i)

In [20]:
if filter_models == "linear_regime":
    # keep only the models that have z_rest > -10 mm
    use_models = []
    for i in range(N_models):
        if metric[i] < 2.5: # q_bar[i][2] < -8 
            use_models.append(i)

In [21]:
if filter_models == "all" or filter_models == "only_default":
    use_models = list(range(N_models))

Hand-select models

In [22]:
# index = 28
# path = join("/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_closed-loop_analysis/adiabatic_ssm_modified_idw_50/050", f"use_models_{index}.pkl")
# with open(path, "rb") as f:
#     use_models = pickle.load(f)

In [23]:
# use_models = [34, 99, 2, 90, 84, 83, 6, 82, 81, 9, 10, 77, 75, 72, 70, 68, 61, 59, 55, 51, 20, 21, 1, 23, 48, 25, 26, 47, 28, 43, 30, 31, 32, 37, 49, 86, 19, 85, 35, 36]

In [24]:
# print(f'Number of used models: {len(use_models)}/{N_models}')
# print(f"USE_MODELS = {use_models}")

Plot the remaining grid points again

In [25]:
if filter_models != "all" and filter_models != "only_default":
    # plt.close("all")
    fig = plt.figure()

    R_linear_part = [r[:, :6] for r in r_coeff]
    eigenvalues_real_part = [np.real(np.linalg.eigvals(r)) for r in R_linear_part]
    metric = np.array([np.linalg.norm(eigenvalues, ord=-np.inf) for eigenvalues in eigenvalues_real_part])

    if INTERPOLATE == "xyz" or INTERPOLATE == "reduced_coords":
        ax = plt.axes(projection="3d")
        if color_by_metric:
            sc = ax.scatter(psi_eq_array[use_models, 0], psi_eq_array[use_models, 1], psi_eq_array[use_models, 2], marker="o", c=metric, vmax=None, cmap='viridis')
        else:
            not_use_models = np.ones(N_models, bool)
            not_use_models[use_models] = 0
            sc = ax.scatter(psi_eq_array[not_use_models, 0], psi_eq_array[not_use_models, 1], psi_eq_array[not_use_models, 2], marker="o", color="darkblue")
            sc = ax.scatter(psi_eq_array[use_models, 0], psi_eq_array[use_models, 1], psi_eq_array[use_models, 2], marker="o", color="tab:orange")
        for i in range(N_models):
            ax.text(psi_eq_array[i, 0], psi_eq_array[i, 1], psi_eq_array[i, 2], str(i), size=10, zorder=1, color='k')
        ax.set_zlabel(r"$x_3$")
    else:
        ax = plt.axes()
        if INTERPOLATE == "xy":
            color = "tab:blue"
        elif INTERPOLATE == "reduced_coords":
            color = "tab:orange"
        if color_by_metric:
            sc = ax.scatter(psi_eq_array[use_models, 0], psi_eq_array[use_models, 1], marker="o", c=metric, vmax=None, cmap="viridis")
        else:
            sc = ax.scatter(psi_eq_array[use_models, 0], psi_eq_array[use_models, 1], marker="o", color=color)
    if INTERPOLATE in  ["xy", "xyz"]:
        ax.set_xlabel(r"$x$ [mm]")
        ax.set_ylabel(r"$y$ [mm]")
    elif INTERPOLATE == "reduced_coords":
        ax.set_xlabel(r"$x_1$")
        ax.set_ylabel(r"$x_2$")
    ax.set_aspect("equal")
    if color_by_metric:
        plt.colorbar(sc, ax=ax, label=r"$||Re(\lambda)||_{\infty}$")
        fig.suptitle("Slowest decay mode of the ROM's linear part")
    fig.show()

Drop all unused models

In [26]:
if filter_models != "all" and filter_models != "only_default":
    # only keep models in use_models
    for i in range(N_models):
        if i not in use_models:
            V[i] = None
            r_coeff[i] = None
            w_coeff[i] = None
            v_coeff[i] = None
            B_r[i] = None
            q_bar[i] = None
            u_bar[i] = None
            psi_eq[i] = None
    V = [V[i] for i in range(len(V)) if V[i] is not None]
    r_coeff = [r_coeff[i] for i in range(len(r_coeff)) if r_coeff[i] is not None]
    w_coeff = [w_coeff[i] for i in range(len(w_coeff)) if w_coeff[i] is not None]
    v_coeff = [v_coeff[i] for i in range(len(v_coeff)) if v_coeff[i] is not None]
    B_r = [B_r[i] for i in range(len(B_r)) if B_r[i] is not None]
    q_bar = [q_bar[i] for i in range(len(q_bar)) if q_bar[i] is not None]
    u_bar = [u_bar[i] for i in range(len(u_bar)) if u_bar[i] is not None]
    psi_eq = [psi_eq[i] for i in range(len(psi_eq)) if psi_eq[i] is not None]
    N_models = len(V)
    print(f'N_models: {N_models}')

Load test trajectory (for now: sum of all trajectories used to regress B matrices)

In [27]:
# test_trajectory_dir = "open-loop"
test_trajectories = []
# for name in tqdm(model_names): # ['origin']: # 
traj_dir = "/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/dataCollection/open-loop_500" # join(model_dir, name, test_trajectory_dir)
(t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, traj_index=0)
y = assemble_observables(z)
print(y.shape)
test_trajectories.append({
        'name': split(traj_dir)[-1],
        't': t,
        'z': z,
        'u': u,
        'y': y
    })

(15, 20001)


Combine into one long trajectory

In [28]:
z_tot = np.hstack([traj['z'] for traj in test_trajectories])
y_tot = np.hstack([traj['y'] for traj in test_trajectories])
u_tot = np.hstack([traj['u'] for traj in test_trajectories])
t_tot = np.arange(z_tot.shape[1]) * DT

In [29]:
print(y_tot[:, :7])
print(z_tot[:, :7])

[[ 1.45404214  1.45404214  1.45404214  1.45404214  1.45404214  2.33910265  3.80858391]
 [-0.30603326 -0.30603326 -0.30603326 -0.30603326 -0.30603326  0.22956691  1.0630584 ]
 [-0.61577293 -0.61577293 -0.61577293 -0.61577293 -0.61577293 -1.12924379 -1.68801492]
 [ 1.45404214  1.45404214  1.45404214  1.45404214  2.33910265  3.80858391  5.76772698]
 [-0.30603326 -0.30603326 -0.30603326 -0.30603326  0.22956691  1.0630584   2.08882514]
 [-0.61577293 -0.61577293 -0.61577293 -0.61577293 -1.12924379 -1.68801492 -2.29402946]
 [ 1.45404214  1.45404214  1.45404214  2.33910265  3.80858391  5.76772698  8.14045726]
 [-0.30603326 -0.30603326 -0.30603326  0.22956691  1.0630584   2.08882514  3.22522734]
 [-0.61577293 -0.61577293 -0.61577293 -1.12924379 -1.68801492 -2.29402946 -2.96579944]
 [ 1.45404214  1.45404214  2.33910265  3.80858391  5.76772698  8.14045726 10.85347754]
 [-0.30603326 -0.30603326  0.22956691  1.0630584   2.08882514  3.22522734  4.40423198]
 [-0.61577293 -0.61577293 -1.12924379 -1.68

Interpolate local models to obtain adiabatic SSM model

In [30]:
interpolation_methods = ["modified_idw", "qp", "nn", "linear", "ct"]
coeff_dict = {
            'w_coeff': w_coeff,
            'V': V,
            'r_coeff': r_coeff,
            'B_r': B_r,
            'u_bar': u_bar,
        }
# if INTERPOLATE in ["xyz", "xy"]:
coeff_dict['q_bar'] = q_bar
if INTERPOLATE == "reduced_coords":
    coeff_dict['x_bar'] = [V[i].T @ np.tile(q_bar[i], 5) for i in range(len(q_bar))]

interpolators = {}
for interpolation_method in interpolation_methods:
    if interpolation_method in ["qp", "linear", "ct"]:
        interp_slice = np.s_[:2]
    else:
        interp_slice = np.s_[:]
    interpolators[interpolation_method] = InterpolatorFactory(interpolation_method, [psi_eq[interp_slice] for psi_eq in psi_eq], coeff_dict).get_interpolator()

In [31]:
display_names = {
    "origin_only": "Origin model",
    "nn": "Nearest neighbor",
    "linear": "Barycentric linear",
    "ct": "Clough-Tocher",
    "idw": "Inverse distance weighting",
    "modified_idw": "Modified IDW",
    "qp": "Quadratic polynomial regression",
    "natural_neighbor": "Natural neighbor",
    "koopman": "Koopman",
    "tpwl": "TPWL",
}

Visualize interpolation landscapes

In [ ]:
if INTERPOLATE in ["xy", "reduced_coords"]:
    nx, ny = (500, 500)
    x = np.linspace(-50, 50, nx)
    y = np.linspace(-50, 50, ny)
    xv, yv = np.meshgrid(x, y)
    # xy = np.vstack([xv.ravel(), yv.ravel()]).T
    coeff = "r_coeff"
    entry = np.s_[0, 0]
    plt.close("all")

    z = {}
    # grad_norm = {}

    for interpolation_method in interpolation_methods:
        print(f"======== {interpolation_method} =========")
        z[interpolation_method] = np.zeros((nx, ny))
        if interpolation_method in ["qp", "linear", "ct"]:
            interp_slice = np.s_[:2]
        else:
            interp_slice = np.s_[:]
        for i in tqdm(range(nx)):
            for j in range(ny):
                if INTERPOLATE == "xy":
                    psi = [xv[i, j], yv[i, j]]
                elif INTERPOLATE == "reduced_coords":
                    psi = np.zeros(6)
                    psi[0:2] = [xv[i, j], yv[i, j]]
                else:
                    psi = None
                z[interpolation_method][i, j] = interpolators[interpolation_method].transform(psi[interp_slice], coeff)[entry]
        # gx, gy = np.gradient(z[interpolation_method], x, y)
        # grad_norm[interpolation_method] = np.sqrt(gx**2 + gy**2)
        # print(f"Maximum gradient norm: {np.nanmax(grad_norm[interpolation_method]):.3f}")


In [ ]:
if INTERPOLATE in ["xy", "reduced_coords"]:
    plt.close("all")
    fig1, axs1 = plt.subplots(1, len(interpolation_methods), figsize=(15, 4), subplot_kw={"projection": "3d"})
    fig2, axs2 = plt.subplots(1, len(interpolation_methods), figsize=(15, 4), sharey=True)

    vmin, vmax = -3.14, -3.08

    for i, interpolation_method in enumerate(interpolation_methods):
        ax1 = axs1[i]
        ax1.plot_surface(xv, yv, z[interpolation_method], cmap="viridis", vmin=vmin, vmax=vmax)
        # xi, yi, zi = psi_eq_array[:, 0], psi_eq_array[:, 1], np.array([r[entry] for r in r_coeff])
        # xi, yi, zi = xi[((np.abs(xi) < 50) & (np.abs(yi) < 50))], yi[((np.abs(xi) < 50) & (np.abs(yi) < 50))], zi[((np.abs(xi) < 50) & (np.abs(yi) < 50))]
        # ax1.plot(xi, yi, zi, color="tab:blue", ls="", marker="o", markersize=12, alpha=1)
        ax1.set_xlabel(r"$x_1$")
        ax1.set_ylabel(r"$x_2$")
        ax1.set_zlabel(r"$R[0, 0]$")
        ax1.set_zlim(vmin, vmax)
        # ax.view_init(90, -90)
        ax1.set_title(display_names[interpolation_method])
        ax1.set_zticks([])
        ax1.set_zticklabels([])
        ax1.set_box_aspect((1, 1, 0.7))
        ax1.set_xlim(-50, 50)
        ax1.set_ylim(-50, 50)
        ax2 = axs2[i]
        im = ax2.contourf(xv, yv, z[interpolation_method], cmap="viridis", levels=20, vmin=vmin, vmax=vmax)
        ax2.plot(psi_eq_array[:, 0], psi_eq_array[:, 1], color="white", ls="", marker="o", markersize=3, alpha=1)
        ax2.set_xlabel(r"$x_1$")
        ax2.set_aspect("equal")
        ax2.set_xlim(-50, 50)
        ax2.set_ylim(-50, 50)
    axs2[0].set_ylabel(r"$x_2$")    
    # fig2.colorbar(im, ax=axs2.ravel().tolist(), shrink=.95, pad=0.02, fraction=1, label=r"$R[0, 0]$")

        # ax2 = axs2[i]
        # im = ax2.imshow(grad_norm[interpolation_method], cmap="viridis", vmin=0, vmax=.1)

        # ax2.set_title(display_names[interpolation_method])
        # ax2.set_xticks([])
        # ax2.set_yticks([])
    # plt.colorbar(im, ax=axs2.ravel().tolist(), shrink=.95, pad=0.02, fraction=1, label="Gradient norm")
    fig1.savefig(join(traj_dir, f"interpolation_landscapes_3d.png"), bbox_inches='tight', dpi=300)
    fig2.savefig(join(traj_dir, f"interpolation_landscapes_2d-contour.png"), bbox_inches='tight', dpi=300)
    fig1.show()
    fig2.show()

Sample finite-horizon predictions to evaluate interpolated models

In [32]:
N_horizon = 5
N_samples = 500

In [33]:
print(z_tot.shape)
np.random.seed(seed=1)
sample_indices = randint(0, z_tot.shape[1], N_samples)

(3, 20001)


In [34]:
import warnings
warnings.filterwarnings('ignore')

z_preds = {}
rmse_samples = {}
xdots = {}

for interpolation_method in tqdm(interpolation_methods, position=0):
    print(f"==================== {interpolation_method} ====================")
    q_samples = []
    rmse_samples[interpolation_method] = []
    xdots[interpolation_method] = []
    advect_times = []
    z_preds[interpolation_method] = []
    z_trues = []

    if interpolation_method == "qp":
        interp_slice = np.s_[:2]
    elif interpolation_method in ["linear", "ct"]:
        interp_slice = np.s_[:2]
    else:
        interp_slice = None

    for i in tqdm(range(N_samples), position=1, leave=True):
        try:
            start_idx = sample_indices[i]
            end_idx = start_idx + N_horizon
            q_samples.append(z_tot[:3, start_idx])
            # advect ASSM to obtain finite-horizon prediction
            t0 = time.time()
            t, _, y_pred, xdot, _, _, _ = utils.advect_adiabaticRD_with_inputs(t_tot[start_idx:end_idx], y_tot[:, start_idx], u_tot[:, start_idx:end_idx],
                                                                            y_target=y_tot[:, start_idx:end_idx], interpolate=INTERPOLATE, interp_slice=interp_slice,
                                                                            interpolator=interpolators[interpolation_method],
                                                                            ROMOrder=ROMOrder, know_target=False)
            t1 = time.time()
            # compute RMSE
            z_pred, z_true = y_pred[-3:, :], y_tot[-3:, start_idx:end_idx]
            rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_true, axis=0)**2))
            rmse_samples[interpolation_method].append(rmse)
            advect_times.append(t1 - t0)
            z_preds[interpolation_method].append(z_pred)
            z_trues.append(z_true)
            xdots[interpolation_method].append(xdot)
        except:
            # RMSE is nan
            rmse_samples[interpolation_method].append(np.nan)
            advect_times.append(np.nan)
            z_preds[interpolation_method].append(np.full((3, N_horizon), np.nan))
            z_trues.append(np.full((3, N_horizon), np.nan))
            xdots[interpolation_method].append(np.full((6, N_horizon), np.nan))
        
    # max_rmse_index = np.argmax(rmse_samples)
    print("avg RMSE:", np.nanmean(rmse_samples[interpolation_method]))
    # print("max RMSE sample idx:", sample_indices[max_rmse_index])
    with open(join(traj_dir, f"{interpolation_method}_n=9_rmse_samples.pkl"), "wb") as f:
        pickle.dump(rmse_samples[interpolation_method], f)
    with open(join(traj_dir, f"{interpolation_method}_n=9_q_samples.pkl"), "wb") as f:
        pickle.dump(q_samples, f)
    with open(join(traj_dir, f"{interpolation_method}_n=9_advect_times.pkl"), "wb") as f:
        pickle.dump(advect_times, f)

  0%|          | 0/5 [00:00<?, ?it/s]

==================== modified_idw ====================


  0%|          | 0/500 [00:00<?, ?it/s]

avg RMSE: 0.720862165797345
==================== qp ====================


  0%|          | 0/500 [00:00<?, ?it/s]

...

avg RMSE: 0.8785582443994367
==================== linear ====================


  0%|          | 0/500 [00:00<?, ?it/s]

avg RMSE: 0.7171938572013488
==================== ct ====================


  0%|          | 0/500 [00:00<?, ?it/s]

avg RMSE: 0.7970956867639717


Plot the random ground-truth trajectory

In [ ]:
plot.predicted_trajectories(np.zeros(z_tot.shape[0]), z_tot, N_samples * [np.full((3, N_horizon), np.nan)], N_samples * [np.full((3, N_horizon), np.nan)], save_path="predicted_trajectories_ground-truth")

Visualize the best and worst predicted trajectories, branching out from random input trajectory

In [ ]:
plt.close("all")
plot.predicted_trajectories(rmse_samples["origin_only"], z_tot, z_preds["origin_only"], z_trues, save_path="predicted_trajectories_origin_only")
plot.predicted_trajectories(rmse_samples["modified_idw"], z_tot, z_preds["modified_idw"], z_trues, save_path="predicted_trajectories_modified_idw")
plot.predicted_trajectories(rmse_samples["qp"], z_tot, z_preds["qp"], z_trues, save_path="predicted_trajectories_qp")

## Baselines

Evaluate prediction accuracy for baselines: Koopman, TPWL

### Koopman

Load Koopman model

In [ ]:
from scipy.io import loadmat
from sofacontrol.baselines.koopman import koopman_utils

In [ ]:
koopman_data = loadmat("/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/koopman_model.mat")['py_data'][0, 0]
raw_model = koopman_data['model']
raw_params = koopman_data['params']
model = koopman_utils.KoopmanModel(raw_model, raw_params)
scaling = koopman_utils.KoopmanScaling(scale=model.scale)

In [ ]:
model.A_d.shape
# model.B_d.shape
# model.V.shape
# model.W.shape
# model.state_dim
# model.C.shape
# zeta_test = np.ones(14)
# lifted_zeta_test = model.lift_data(*zeta_test)
# model.B_d

Advect Koopman model and evaluate model accuracy

In [ ]:
z_tot_koopman = (z_tot.T + rest_q[3*TIP_NODE:3*(TIP_NODE+1)]).T
z_tot_koopman = scaling.scale_down(y=z_tot_koopman.T).T
u_tot_koopman = scaling.scale_down(u=utils.delayEmbedding(u_tot, up_to_delay=1, embed_coords=list(range(8)))[:8, :].T).T
y_tot_koopman = np.vstack([z_tot_koopman, utils.delayEmbedding(z_tot_koopman, up_to_delay=1)[:3, :], u_tot_koopman])

rmse_samples = []
xdots = []
advect_times = []
z_preds = []
z_trues = []
q_samples = []

for i in tqdm(range(N_samples), position=1, leave=True):
    start_idx = sample_indices[i]
    end_idx = start_idx + N_horizon
    q_samples.append(z_tot[:3, start_idx])
    t0 = time.time()
    # advect Koopman to obtain finite-horizon prediction
    t, y, u = t_tot[start_idx:end_idx], y_tot_koopman[:, start_idx:end_idx], u_tot_koopman[:, start_idx:end_idx]
    y0 = y[:, 0]
    N = len(t)-1
    x = np.full((model.A_d.shape[0], N+1), np.nan)
    xdot = np.full((model.A_d.shape[0], N), np.nan)
    y_pred = np.full((len(y0), N+1), np.nan)
    y_pred[:, 0] = y0
    # advect for horizon N
    for i in range(N):
        # lift the observables
        if i == 0:
            x[:, i] = model.lift_data(*y_pred[:, i])
        # compute the Koopman prediction
        # xdot[:, i]
        x[:, i+1] = model.A_d @ x[:, i] + model.B_d @ u[:, i]
        # forward Euler: x[i+1] = x[i] + dt * xdot[i]
        # x[:, i+1] = x[:, i] + DT * xdot[:, i]
        y_pred[:, i+1] = np.concatenate([model.C @ x[:, i+1], y_pred[:3, i], u[:, i]])
    t1 = time.time()
    # compute RMSE
    z_pred = (scaling.scale_up(y=y_pred[:3, :].T) - rest_q[3*TIP_NODE:3*(TIP_NODE+1)]).T
    z_true = z_tot[:, start_idx:end_idx]
    rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_true, axis=0)**2))
    rmse_samples.append(rmse)
    advect_times.append(t1 - t0)
    z_preds.append(z_pred)
    z_trues.append(z_true)
    xdots.append(xdot)

print("avg RMSE:", np.nanmean(rmse_samples))
# print("max RMSE sample idx:", sample_indices[max_rmse_index])
with open(join(traj_dir, f"koopman_rmse_samples.pkl"), "wb") as f:
    pickle.dump(rmse_samples, f)
with open(join(traj_dir, f"koopman_q_samples.pkl"), "wb") as f:
    pickle.dump(q_samples, f)
with open(join(traj_dir, f"koopman_advect_times.pkl"), "wb") as f:
    pickle.dump(advect_times, f)

print(rmse_samples)

In [ ]:
plot.predicted_trajectories(rmse_samples, z_tot, z_preds, z_trues, save_path="predicted_trajectories_koopman")

### TPWL

Load TPWL model

In [ ]:
from sofacontrol.tpwl import tpwl, tpwl_config
from sofacontrol.measurement_models import linearModel, MeasurementModel

# Specify a measurement and output model
cov_q = 0.0 * np.eye(3 * len([TIP_NODE]))
cov_v = 0.0 * np.eye(3 * len([TIP_NODE]))
measurement_model = linearModel(nodes=[TIP_NODE], num_nodes=N_NODES)
output_model = MeasurementModel(nodes=[TIP_NODE], num_nodes=N_NODES, pos=True, vel=True, S_q=cov_q, S_v=cov_v)
# Load and configure the TPWL model from data saved
tpwl_model_file = "/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/tpwl_model_snapshots.pkl"
config = tpwl_config.tpwl_dynamics_config()
model = tpwl.TPWLATV(data=tpwl_model_file, params=config.constants_sim, Hf=output_model.C,
                     Cf=measurement_model.C)
model.pre_discretize(dt=DT)

In [ ]:
print(measurement_model.C.shape)
print(model.C.shape)

Import trajectories (pos and vel) of all FEM nodes

In [ ]:
# test_trajectory_dir = "open-loop"
test_trajectories = []
# for name in tqdm(model_names): # ['origin']: # 
traj_dir = "/home/jonas/Projects/stanford/soft-robot-control/examples/trunk/dataCollection/open-loop_500" # join(model_dir, name, test_trajectory_dir)
(t, qv), u = utils.import_pos_data(data_dir=traj_dir,
                                  rest_file=rest_file,
                                  output_node="all", return_inputs=True, return_velocity=True, traj_index=0)

test_trajectories.append({
        'name': split(traj_dir)[-1],
        't': t,
        'q': qv[:3*N_NODES, :],
        'v': qv[3*N_NODES:, :],
        'u': u,
    })

q_tot = np.hstack([traj['q'] for traj in test_trajectories])
v_tot = np.hstack([traj['v'] for traj in test_trajectories])
u_tot = np.hstack([traj['u'] for traj in test_trajectories])
t_tot = np.arange(z_tot.shape[1]) * DT

In [ ]:
print("q_tot.shape:", q_tot.shape)
print("v_tot.shape:", v_tot.shape)

Advect TWPL model and evaluate model accuracy

In [ ]:
print(model.H.shape)
print(model.C.shape)

In [ ]:
rmse_samples = []
xdots = []
advect_times = []
z_preds = []
z_true = []
q_samples = []

for j in tqdm(range(N_samples), position=1, leave=True):
    start_idx = sample_indices[j]
    end_idx = start_idx + N_horizon
    q_samples.append(z_tot[:3, start_idx])
    t0 = time.time()
    t, q, v, u = t_tot[start_idx:end_idx], q_tot[:, start_idx:end_idx], v_tot[:, start_idx:end_idx], u_tot[:, start_idx:end_idx]
    N = len(t)-1
    xf = np.vstack([v, q]) + model.rom.x_ref[:, None]
    # print(xf.shape)
    x0 = model.rom.compute_RO_state(xf=xf[:, 0])
    x = np.full((model.state_dim, N+1), np.nan)
    xdot = np.full((model.state_dim, N), np.nan)
    x[:, 0] = x0
    z0 = model.x_to_zy(x0, z=True)
    # print(z0)
    # print(z_tot[:, start_idx])
    # advect for horizon N
    for i in range(N):
        # compute the TPWL prediction
        A, B, d = model.get_jacobians(x=x[:, i], u=u[:, i], dt=DT)
        # xdot[i] = R(x[i]) + B[i] @ (u[i] - u_bar[i])
        # xdot[:, i] = A @ x[:, i] + B @ u[:, i] + d
        # print(np.linalg.eig(A))
        # forward Euler: x[i+1] = x[i] + dt * xdot[i]
        x[:, i+1] = A @ x[:, i] + B @ u[:, i] + d
        # x[:, i+1] = x[:, i] + DT * xdot[:, i]
    t1 = time.time()
    # compute RMSE
    z_pred = model.x_to_zy(x.T, z=True).T[3:, :]
    z_true = z_tot[:, start_idx:end_idx]
    # print(z_pred.shape)
    # print(z_true.shape)
    rmse = np.sqrt(np.mean(np.linalg.norm(z_pred - z_true, axis=0)**2))
    rmse_samples.append(rmse)
    advect_times.append(t1 - t0)
    z_preds.append(z_pred)
    z_trues.append(z_true)
    xdots.append(xdot)

print("avg RMSE:", np.nanmean(rmse_samples))
print("min RMSE:", np.nanmin(rmse_samples), "max RMSE:", np.nanmax(rmse_samples))
# print("max RMSE sample idx:", sample_indices[max_rmse_index])
with open(join(traj_dir, f"tpwl_rmse_samples.pkl"), "wb") as f:
    pickle.dump(rmse_samples, f)
with open(join(traj_dir, f"tpwl_q_samples.pkl"), "wb") as f:
    pickle.dump(q_samples, f)
with open(join(traj_dir, f"tpwl_advect_times.pkl"), "wb") as f:
    pickle.dump(advect_times, f)

In [ ]:
plot.predicted_trajectories(rmse_samples, z_tot, z_preds, z_trues, save_path="predicted_trajectories_tpwl")

Plot prediction accuracy maps for all the different interpolation methods

In [35]:
plt.close("all")

In [40]:
import copy

interpolation_methods = ["linear", "ct", "ct_n=9"] # ["modified_idw", "qp", "nn"] # 
baselines = []
methods = interpolation_methods + baselines

display_names_all_adiabatic = copy.copy(display_names)
display_names_all_adiabatic["ct"] = r"Clough-Tocher ($N_{grid}=109$)"
display_names_all_adiabatic["ct_n=9"] = r"Clough-Tocher ($N_{grid}=9$)"

show_advect_times = False

fig, axs = plt.subplots(3 + show_advect_times, len(methods),
                        figsize=(4*len(methods), (12 if show_advect_times else 10)),
                        height_ratios=([4, 3, 2, 2] if show_advect_times else [5, 2, 1]),
                        sharey='row', sharex='row', layout="compressed")
for i, method in enumerate(methods):
    with open(join(traj_dir, f"{method}_rmse_samples.pkl"), "rb") as f:
        rmse_samples = np.array(pickle.load(f))
    with open(join(traj_dir, f"{method}_q_samples.pkl"), "rb") as f:
        q_samples = np.stack(pickle.load(f))
    with open(join(traj_dir, f"{method}_advect_times.pkl"), "rb") as f:
        advect_times = np.array(pickle.load(f))
    colorbar = (i == len(methods) - 1)
    if i == 0:
        ylabels = [r"$y$ [mm]", r"$z$ [mm]"]
    else:
        ylabels = ["", ""]
    plot.prediction_accuracy_map(q_samples[:, [0, 1]], rmse_samples, vmin=0., vmax=4., ax=axs[0, i], colorbar=colorbar, ylabel=ylabels[0], cax=axs[:, :], show=False)
    plot.prediction_accuracy_map(np.array([q_samples[:, 0], -q_samples[:, 2]]).T, rmse_samples, vmin=0., vmax=4., ax=axs[1, i], colorbar=False, ylabel=ylabels[1], cax=axs[:, :], show=False)
    plot.boxplot(rmse_samples, ax=axs[2, i], show=False, xlabel="RMSE [mm]")
    if show_advect_times:
        plot.boxplot(advect_times, ax=axs[2, i], show=False, xlabel="Advect time [s]")
    axs[0, i].set_title(display_names_all_adiabatic[method])
    # if i > 0:
    #     axs[0, i].set_ylabel("")
fig.savefig(join(traj_dir, f"prediction_accuracy_all-adiabatic-B.png"), bbox_inches='tight', dpi=300)
fig.savefig(join(traj_dir, f"prediction_accuracy_all-adiabatic-B.eps"), bbox_inches='tight', dpi=300)
fig.show()

In [ ]:
interpolation_methods = ["modified_idw", "origin_only"]
baselines = ["tpwl"]
methods = interpolation_methods + baselines

show_advect_times = False

fig, axs = plt.subplots(3 + show_advect_times, len(methods),
                        figsize=(4*len(methods), (12 if show_advect_times else 10)),
                        height_ratios=([4, 3, 2, 2] if show_advect_times else [5, 2, 1]),
                        sharey='row', sharex='row', layout="compressed")
for i, method in enumerate(methods):
    with open(join(traj_dir, f"{method}_rmse_samples.pkl"), "rb") as f:
        rmse_samples = np.array(pickle.load(f))
    with open(join(traj_dir, f"{method}_q_samples.pkl"), "rb") as f:
        q_samples = np.stack(pickle.load(f))
    with open(join(traj_dir, f"{method}_advect_times.pkl"), "rb") as f:
        advect_times = np.array(pickle.load(f))
    colorbar = (i == len(methods) - 1)
    if i == 0:
        ylabels = [r"$y$ [mm]", r"$z$ [mm]"]
    else:
        ylabels = ["", ""]
    plot.prediction_accuracy_map(q_samples[:, [0, 1]], rmse_samples, vmin=0., vmax=4., ax=axs[0, i], colorbar=colorbar, ylabel=ylabels[0], cax=axs[:, :], show=False)
    plot.prediction_accuracy_map(np.array([q_samples[:, 0], -q_samples[:, 2]]).T, rmse_samples, vmin=0., vmax=4., ax=axs[1, i], colorbar=False, ylabel=ylabels[1], cax=axs[:, :], show=False)
    plot.boxplot(rmse_samples, ax=axs[2, i], show=False, xlabel="RMSE [mm]")
    if show_advect_times:
        plot.boxplot(advect_times, ax=axs[2, i], show=False, xlabel="Advect time [s]")
    axs[0, i].set_title(display_names[method])
    # if i > 0:
    #     axs[0, i].set_ylabel("")
fig.savefig(join(traj_dir, f"prediction_accuracy.png"), bbox_inches='tight', dpi=300)
fig.savefig(join(traj_dir, f"prediction_accuracy.eps"), bbox_inches='tight', dpi=300)
fig.show()

In [ ]:
interpolation_methods = ["modified_idw", "origin_only", ] # ["origin_only", "idw", "modified_idw"] # , "qp"] # ["origin_only", "linear", "nn", "qp"] # , "tps", "ls", "qp"]
baselines = ["koopman"]
methods = interpolation_methods + baselines

show_advect_times = False

fig, axs = plt.subplots(3 + show_advect_times, len(methods),
                        figsize=(4*len(methods), (12 if show_advect_times else 10)),
                        height_ratios=([4, 3, 2, 2] if show_advect_times else [5, 2, 1]),
                        sharey='row', sharex='row', layout="compressed")
for i, method in enumerate(methods):
    with open(join(traj_dir, f"{method}_rmse_samples.pkl"), "rb") as f:
        rmse_samples = np.array(pickle.load(f))
    with open(join(traj_dir, f"{method}_q_samples.pkl"), "rb") as f:
        q_samples = np.stack(pickle.load(f))
    with open(join(traj_dir, f"{method}_advect_times.pkl"), "rb") as f:
        advect_times = np.array(pickle.load(f))
    colorbar = (i == len(methods) - 1)
    if i == 0:
        ylabels = [r"$y$ [mm]", r"$z$ [mm]"]
    else:
        ylabels = ["", ""]
    plot.prediction_accuracy_map(q_samples[:, [0, 1]], rmse_samples, vmin=0., vmax=50., ax=axs[0, i], colorbar=colorbar, ylabel=ylabels[0], cax=axs[:, :], show=False) # 4.
    plot.prediction_accuracy_map(np.array([q_samples[:, 0], -q_samples[:, 2]]).T, rmse_samples, vmin=0., vmax=50., ax=axs[1, i], colorbar=False, ylabel=ylabels[1], cax=axs[:, :], show=False) # 4.
    plot.boxplot(rmse_samples, ax=axs[2, i], show=False, xlabel="RMSE [mm]")
    if show_advect_times:
        plot.boxplot(advect_times, ax=axs[2, i], show=False, xlabel="Advect time [s]")
    axs[0, i].set_title(display_names[method])
    # if i > 0:
    #     axs[0, i].set_ylabel("")
fig.savefig(join(traj_dir, f"prediction_accuracy_ssm_and_koopman.png"), bbox_inches='tight', dpi=300)
fig.savefig(join(traj_dir, f"prediction_accuracy_ssm_and_koopman.eps"), bbox_inches='tight', dpi=300)
fig.show()

In [ ]:
interpolation_methods = []
baselines = ["koopman"]
methods = interpolation_methods + baselines

show_advect_times = False

fig, axs = plt.subplots(3 + show_advect_times, len(methods),
                        figsize=(4*len(methods), (12 if show_advect_times else 10)),
                        height_ratios=([4, 3, 2, 2] if show_advect_times else [5, 2, 1]),
                        sharey='row', sharex='row', layout="compressed")
axs = np.atleast_2d(axs).T
for i, method in enumerate(methods):
    with open(join(traj_dir, f"{method}_rmse_samples.pkl"), "rb") as f:
        rmse_samples = np.array(pickle.load(f))
    with open(join(traj_dir, f"{method}_q_samples.pkl"), "rb") as f:
        q_samples = np.stack(pickle.load(f))
    with open(join(traj_dir, f"{method}_advect_times.pkl"), "rb") as f:
        advect_times = np.array(pickle.load(f))
    colorbar = (i == len(methods) - 1)
    if i == 0:
        ylabels = [r"$y$ [mm]", r"$z$ [mm]"]
    else:
        ylabels = ["", ""]
    plot.prediction_accuracy_map(q_samples[:, [0, 1]], rmse_samples, vmin=0., vmax=50., ax=axs[0, i], colorbar=colorbar, ylabel=ylabels[0], cax=axs[:, :], show=False)
    plot.prediction_accuracy_map(np.array([q_samples[:, 0], -q_samples[:, 2]]).T, rmse_samples, vmin=0., vmax=50., ax=axs[1, i], colorbar=False, ylabel=ylabels[1], cax=axs[:, :], show=False)
    plot.boxplot(rmse_samples, ax=axs[2, i], show=False, xlabel="RMSE [mm]")
    if show_advect_times:
        plot.boxplot(advect_times, ax=axs[2, i], show=False, xlabel="Advect time [s]")
    axs[0, i].set_title(display_names[method])
    # if i > 0:
    #     axs[0, i].set_ylabel("")
fig.savefig(join(traj_dir, f"prediction_accuracy_koopman.png"), bbox_inches='tight', dpi=300)
fig.savefig(join(traj_dir, f"prediction_accuracy_koopman.eps"), bbox_inches='tight', dpi=300)
fig.show()